# Подготовка массивов данных для обучения ИНС

1. Загрузка полного массива данных
2. Формирование косинусовых векторов для ИНС директора
3. Формирование прецедентов для ИНС распознавателя

## Загрузка полного массива данных

In [1]:
_DIR_ = 'D:/DNN/cross5_rank1000/L_4/'
LIMIT = 50000
DIM = 5
ETA = 20
PP_K = 2*ETA*(DIM - 1) + 1
print(PP_K)

161


In [2]:
import numpy as np

_in1 = np.load(_DIR_ + 'dataset100000_face1_1.npy')
_in2 = np.load(_DIR_ + 'dataset100000_face1_2.npy')
_in3 = np.load(_DIR_ + 'dataset100000_face1_3.npy')
dataset = np.vstack((_in1, _in2, _in3))
#dataset[:,0:PP_K] -> distances
#dataset[:,PP_K] -> face
#dataset[:,PP_K + 1:PP_K + 1 + DIM] -> cosine coordinates
#dataset[:,PP_K + 1 + DIM:PP_K + 1 + 2*DIM] -> face count
print(len(dataset))

70650


## Формирование косинусовых векторов для ИНС директора 1 (без нулей)

Поместим в `face1_yes` строчки, в которых поле `face` равно `1.`. Движение возможно только по одиночной гиперплоскости.

Сохраним в `test` поля, содержащие образ рецептивного поля, а в `answers` - косинусовые координаты.

Объединим `test` и `answers` в массив `face_vec`. Перемешаем строчки и сохраним в `face_vec.npy`.

## Формирование косинусовых векторов для ИНС директора 2 (без нулей)

Поместим в `face1_yes` строчки, в которых поле `face` равно `1.`. Движение возможно только по гиперплоскости.

In [3]:
face1_yes = dataset[np.in1d(dataset[:, PP_K], np.asarray([1.]))]
#face1_yes = face1_yes[np.in1d(face1_yes[:, PP_K + 1 + DIM + 1], np.asarray([1.]))]
print(face1_yes[:10, PP_K + 1 + DIM + 0]) # Проверка столбца face_count[0] (содержит число многообразий уровня 1, по которым возможно движение)
print(len(face1_yes))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
70650


Сохраним в `test` поля, содержащие образ рецептивного поля, а в `answers` - косинусовые координаты.

In [4]:
test = face1_yes[:,:PP_K]
answers = face1_yes[:,PP_K + 1:PP_K + 1 + DIM]
#answers[:,0] = 0.
print(answers[-5:])

[[-1.14030673e-14  1.00000000e+00 -4.57681704e-15 -3.83071792e-15
   0.00000000e+00]
 [ 3.60213786e-15 -7.70308567e-02 -7.39650123e-02  5.98821010e-01
   7.93730950e-01]
 [ 4.70178957e-15 -4.27352162e-01  9.04085245e-01 -3.30594579e-16
   1.50604197e-15]
 [ 3.92878090e-14  6.17828322e-01  6.67407777e-01  4.11823596e-01
  -5.70644397e-02]
 [ 9.95260793e-15 -2.52982213e-01 -2.63312235e-01 -5.88784058e-01
   7.21110255e-01]]


Объединим `test` и `answers` в массив `face_vec`. Перемешаем строчки и сохраним в `face_vec.npy`.

In [5]:
face1_vec = np.hstack((test,answers))
np.random.shuffle(face1_vec)
print(face1_vec[:5,-5:])
np.save(_DIR_ + 'L4_vec.npy', face1_vec)

[[ 2.87193748e-15  1.00000000e+00  5.38488278e-16  3.36555174e-16
  -1.03210253e-15]
 [-1.18996364e-14  5.43916157e-01 -8.73599970e-02 -4.26682872e-01
   7.17262276e-01]
 [-1.43442473e-15  5.88457652e-01  6.81097556e-01 -2.58242761e-01
   3.50905096e-01]
 [-1.00531923e-14 -5.38109442e-01 -7.10987264e-01  4.08686991e-01
   1.94705631e-01]
 [-2.86454933e-15 -2.78850094e-01 -2.16366645e-01 -8.18596907e-01
   4.53130448e-01]]


## Формирование косинусовых векторов для ИНС директора (включая нули)

## Формирование прецедентов для ИНС распознавателя

Поместим в `face1_yes` строчки, в которых поле `face` равно `2.` Движение возможно только по ребрам. Причем, по единственному ребру.

Поместим в `face1_no` строчки, в которых поле `face` не равно `2.`.

Поместим в `idx` случайный набор индексов. По этим индексам выпишем в `test` случайный набор строк из `face1_yes`. Сгенерируем столбец правильных ответов, состоящий из единиц и поместим его в `answers`. Объединив `test` и `answers`, перезапишем в `face1_yes` набор данных, где первые PP_K столбцов содержат образ рецептивного поля (набор смещений), а последний столбец правильные ответы (единицы).

Поместим в `idx` случайный набор индексов. По этим индексам выпишем в `test` случайный набор строк из `face1_no`. Сгенерируем столбец правильных ответов, состоящий из нулей и поместим его в `answers`. Объединив `test` и `answers`, перезапишем в `face1_no` набор данных, где первые PP_K столбцов содержат образ рецептивного поля (набор смещений), а последний столбец правильные ответы (нули).

Объединим и поместим в `face1` наборы положительных прецедентов `face1_yes` и отрицательных прецедентов `face1_no`. Перемешаем их и сохраним в файл `face_def.npy`.